In [1]:
%pip install opencv-python
import cv2
import numpy as np

proto_file = 'models/colorization_deploy_v2.prototxt'
model_file = 'models/colorization_release_v2.caffemodel'
pts_file = 'models/pts_in_hull.npy'

net = cv2.dnn.readNetFromCaffe(proto_file, model_file)
pts = np.load(pts_file)

Note: you may need to restart the kernel to use updated packages.


In [2]:
class8 = net.getLayerId("class8_ab")
conv8 = net.getLayerId("conv8_313_rh")

pts = pts.transpose().reshape(2, 313, 1, 1)
net.getLayer(class8).blobs = [pts.astype("float32")]
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype="float32")]


In [3]:
images= cv2.imread('images/WhatsApp Image 2025-10-27 at 23.55.28_c6f5b412.jpg')


In [4]:
# Load your black & white image (try file, fallback to existing 'images' variable)
image = cv2.imread('images/bw_photo.jpg')
if image is None:
	try:
		image = images.copy()
		print("Loaded fallback 'images' variable because 'images/bw_photo.jpg' was not found.")
	except NameError:
		raise FileNotFoundError(
			"Could not load 'images/bw_photo.jpg' and no 'images' variable is available."
		)

# Convert the image’s pixel values to float in range [0, 1]
scaled = image.astype("float32") / 255.0

# Convert from BGR (default OpenCV format) to LAB color space
lab = cv2.cvtColor(scaled, cv2.COLOR_BGR2LAB)


Loaded fallback 'images' variable because 'images/bw_photo.jpg' was not found.


In [5]:
L= lab[:, :, 0]
L -= 50
L_resiged= cv2.resize(L, (224, 224))

In [6]:
net.setInput(cv2.dnn.blobFromImage(L_resiged))
ab = net.forward()[0, :, :, :].transpose((1, 2, 0))
ab = cv2.resize(ab, (image.shape[1], image.shape[0]))

In [7]:
L = lab[:, :, 0]
color_lab = np.concatenate((L[:, :, np.newaxis], ab), axis=2)
color_bgr = cv2.cvtColor(color_lab, cv2.COLOR_LAB2BGR)
color_bgr = np.clip(color_bgr, 0, 1)
color_bgr = (255 * color_bgr).astype("uint8")


In [ ]:
# Merge L channel with predicted a and b channels
ab_upscaled = cv2.resize(ab, (image.shape[1], image.shape[0]))
lab_output = np.concatenate((lab[:, :, [0]], ab_upscaled), axis=2)

# Convert LAB -> BGR
colorized = cv2.cvtColor(lab_output, cv2.COLOR_LAB2BGR)

# Clip and scale to 0–255
colorized = np.clip(colorized, 0, 1)
colorized = (255 * colorized).astype("uint8")

# Display
cv2.imshow("Original", image)
cv2.imshow("Colorized", colorized)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Save
cv2.imwrite("images/colorized_image.jpg", colorized)

